In [1]:
######## Find Optimal Decision Boundary Threshold for Models

# Loads model's weights
# Using validation dataset, find optimal threshold
# Test final accuracy on test dataset

Import All Required Modules

In [1]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
print("# === STEP 0: Imports: Starting ...")
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
print("# === STEP 0: Imports: Complete")

# === STEP 0: Imports: Starting ...


2025-01-21 16:19:38.319851: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# === STEP 0: Imports: Complete


In [2]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...")
## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_000.yaml'
yaml_path1 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_001.yaml'
yaml_paths = [yaml_path0, yaml_path1]

checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete")

# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...
# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete


Load Val, and Test Datasets into Memory

In [3]:
import pickle
import os

# File path to load train, test, and val datasets
train_test_val_dataset_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v01"


test_dataset_path = train_test_val_dataset_path + "/test.pkl"

with open(test_dataset_path, "rb") as f:
    test_dataset = pickle.load(f)
    print("Test dataset leaded to memory")

Test dataset leaded to memory


Load Models

In [4]:
## For all YAMLS, get their model's signatures
import yaml

yaml_path_list = [yaml_path1]
signatures = []

for yaml_file in yaml_path_list:
    with open(yaml_file, "r") as file:
        config_data = yaml.safe_load(file)

    # Save model signature 
    signatures.append(config_data.get("model", {}).get("model_signature"))

## Using model's signatures, get model's weights path
weights_paths =[]

for signature in signatures:
    base_path = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/z-model_binaries/model_v0.1/models/'
    base_file_name = '/model_weights_'

    directory = base_path + signature
    file_name = base_file_name + signature + ".pth"

    final_file_path = directory + file_name

    weights_paths.append(final_file_path) 

In [5]:
# === STEP 4: MODEL MANAGER
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)
models = model_manager.load_models([yaml_path1], weights_paths)
print("Models successfully loaded ")

Models successfully loaded 


Predict on Test Set

In [6]:
predictions = []
for model in models:
    predictions.append(model.predict(test_dataset.examples, return_target_labels=True))

total_examples = 0
correct_predictions = 0

for prediction in predictions:
    for row in prediction.itertuples(index=False):
        total_examples += 1

        # Compare predictions and target labels for the current feature
        if (row.final_score_A > row.final_score_B and row.target_final_score_A > row.target_final_score_B) or \
        (row.final_score_A < row.final_score_B and row.target_final_score_A < row.target_final_score_B):
            correct_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_examples
    print(f"Accuracy across all features on test dataset: {accuracy}")

Accuracy across all features on test dataset: 0.5789887346083311
